# Use Case: AI-Powered Support Agent (Chained Execution)

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
import getpass

In [2]:
openai_api_key = getpass.getpass(prompt='Openai Api Key')

Openai Api Key ········


In [3]:
def rule_based_agent(query: str):
    """Handles predefined customer queries, else passes to the next agent."""
    rules = {
        "password reset": "To reset your password, click 'Forgot Password' on the login page.",
        "refund policy": "Our refund policy allows returns within 30 days with a receipt.",
        "cancel subscription": "To cancel, go to account settings and click 'Cancel Subscription'."
    }
    
    for keyword, response in rules.items():
        if keyword in query.lower():
            return {"status": "resolved", "response": response}

    return {"status": "unresolved", "query": query}  # Pass to next agent


In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5, openai_api_key=openai_api_key)

def sentiment_analysis_agent(input_data):
    """Analyzes sentiment of the query before sending to AI response agent."""
    if input_data["status"] == "resolved":  # If rule-based agent handled it, return early
        return input_data

    query = input_data["query"]
    messages = [
        SystemMessage(content="Classify the following text as Positive or Negative sentiment."),
        HumanMessage(content=query)
    ]
    
    sentiment = llm(messages).content.strip()  # Returns 'Positive' or 'Negative'
    
    return {"status": "unresolved", "query": query, "sentiment": sentiment}


In [6]:
def ml_agent(input_data):
    """Uses GPT-4 for intelligent responses if the rule-based system couldn't resolve it."""
    if input_data["status"] == "resolved":  # If already resolved, return early
        return input_data

    query = input_data["query"]
    sentiment = input_data.get("sentiment", "Neutral")

    messages = [
        SystemMessage(content=f"You are a customer support AI. The user sentiment is {sentiment}."),
        HumanMessage(content=query)
    ]
    
    response = llm(messages).content  # AI-generated response
    
    return {"status": "resolved", "response": response}


In [7]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

def web_search_agent(input_data):
    """Searches the web if GPT-4 fails to generate an answer."""
    if input_data["status"] == "resolved":
        return input_data

    query = input_data["query"]
    search_result = search.run(query)
    
    return {"status": "resolved", "response": f"Here's what I found online: {search_result}"}


In [16]:
def customer_support_pipeline(query):
    """Passes the query sequentially through all agents."""
    print(query)
    # Step 1: Rule-Based Agent
    step_1 = rule_based_agent(query)
    print("Step1")
    print(step_1)
    if step_1["status"] == "resolved":
        return step_1["response"]

    # Step 2: Sentiment Analysis
    step_2 = sentiment_analysis_agent(step_1)
    print("Step2")
    print(step_2)
    # Step 3: LLM Agent (GPT-4)
    step_3 = ml_agent(step_2)
    print("Step3")
    print(step_3)
    if step_3["status"] == "resolved":
        return step_3["response"]

    # Step 4: Web Search (Only if LLM fails)
    step_4 = web_search_agent(step_3)
    print("Step4")
    print(step_4)
    return step_4["response"]


In [19]:
# Rule-based response
print("\nQuery 1:", customer_support_pipeline("password reset"))

# # Sentiment Analysis + AI Response
# print("\nQuery 2:", customer_support_pipeline("This service is awful! I need a refund now!"))

# # Complex Query (GPT-4)
# print("\nQuery 3:", customer_support_pipeline("Tell me about the best features of your premium plan."))

# # Unknown Query (Uses Web Search)
# print("\nQuery 4:", customer_support_pipeline("What are the latest offers on your plans?"))


password reset
Step1
{'status': 'resolved', 'response': "To reset your password, click 'Forgot Password' on the login page."}

Query 1: To reset your password, click 'Forgot Password' on the login page.


In [20]:
# Sentiment Analysis + AI Response
print("\nQuery 2:", customer_support_pipeline("This service is awful! I need a refund now!"))

This service is awful! I need a refund now!
Step1
{'status': 'unresolved', 'query': 'This service is awful! I need a refund now!'}
Step2
{'status': 'unresolved', 'query': 'This service is awful! I need a refund now!', 'sentiment': 'Negative'}
Step3
{'status': 'resolved', 'response': "I'm really sorry to hear about your dissatisfaction with our service. It's our priority to ensure that our customers are happy and satisfied. I'll do my best to help you. Could you please provide me with your order number or any other details so I can look into the issue for you?"}

Query 2: I'm really sorry to hear about your dissatisfaction with our service. It's our priority to ensure that our customers are happy and satisfied. I'll do my best to help you. Could you please provide me with your order number or any other details so I can look into the issue for you?
